# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import time
import json

from PIL import Image

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers'
trng_data_dir = data_dir + '/train'
valn_data_dir = data_dir + '/valid'
tstg_data_dir = data_dir + '/test'

In [ ]:
# Define transforms for the training, validation, and testing sets
normalised_transform = transforms.Normalize([0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])

trng_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      normalised_transform])

valn_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      normalised_transform])

tstg_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      normalised_transform])


# Load the datasets with ImageFolder
trng_dataset = datasets.ImageFolder(trng_data_dir, 
                                    transform=trng_transforms)

valn_dataset = datasets.ImageFolder(valn_data_dir, 
                                    transform=valn_transforms)

tstg_dataset = datasets.ImageFolder(tstg_data_dir, 
                                    transform=tstg_transforms)


# Define the dataloaders using the image datasets and trainforms
trng_dataloader = torch.utils.data.DataLoader(trng_dataset, 
                                              batch_size=64, 
                                              shuffle=True)

valn_dataloader = torch.utils.data.DataLoader(valn_dataset,
                                              batch_size=64,
                                              shuffle=True)

tstg_dataloader = torch.utils.data.DataLoader(tstg_dataset,
                                              batch_size=64,
                                              shuffle=True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
with open('cat_to_name.json', 'r') as file:
    cat_to_name = json.load(file)

len(cat_to_name)

In [ ]:
def create_idx_to_cat(cat_to_idx):
    return {val: key for key, val in cat_to_idx.items()}

idx_to_cat = create_idx_to_cat(trng_dataloader.dataset.class_to_idx)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

<font color='red'>**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.</font>

In [ ]:
# Use the GPU if it's available
CPU_DEVICE = 'cpu'
GPU_DEVICE = 'cuda'

device = torch.device(GPU_DEVICE if torch.cuda.is_available() else CPU_DEVICE)

In [ ]:
def create_model_densenet121(hidden_layer_size=256, lr=0.003):
    model = models.densenet121(pretrained=True)

    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False
        
    model.classifier = nn.Sequential(nn.Linear(1024, hidden_layer_size),
                                     nn.ReLU(),
                                     nn.Dropout(0.2),
                                     nn.Linear(hidden_layer_size, 102),
                                     nn.LogSoftmax(dim=1))

    criterion = nn.NLLLoss()

    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr)

    return model, optimizer, criterion

In [ ]:
def create_model_resnet101(hidden_layer_size=256, lr=0.003):
    model = models.resnet101(pretrained=True)

    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False
        
    model.fc = nn.Sequential(nn.Linear(2048, hidden_layer_size),
                             nn.ReLU(),
                             nn.Dropout(0.2),
                             nn.Linear(hidden_layer_size, 102),
                             nn.LogSoftmax(dim=1))

    criterion = nn.NLLLoss()

    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.fc.parameters(), lr)

    return model, optimizer, criterion

In [ ]:
def create_model_vgg11(hidden_layer_size=256, lr=0.003):
    model = models.vgg11(pretrained=True)

    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False
        
    model.classifier[-1] = nn.Sequential(nn.Linear(4096, hidden_layer_size),
                                         nn.ReLU(),
                                         nn.Dropout(0.2),
                                         nn.Linear(hidden_layer_size, 102),
                                         nn.LogSoftmax(dim=1))

    criterion = nn.NLLLoss()

    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr)

    return model, optimizer, criterion

In [ ]:
max_epochs = 15
steps = 0
hidden_layer_size = 256
lr = 0.0005

model, optimizer, criterion = create_model_densenet121(hidden_layer_size, lr)
model.to(device)

trng_losses, valn_losses = [], []
for epoch in range(max_epochs):

    start_time = time.time()
    
    total_trng_loss = 0
    model.train()
    for images, exp_results in trng_dataloader:
        images, exp_results = images.to(device), exp_results.to(device)
        optimizer.zero_grad()
        
        log_pred_results = model(images)
        loss = criterion(log_pred_results, exp_results)
        total_trng_loss += loss.item()

        loss.backward()
        optimizer.step()
        
    total_valn_loss = 0
    total_correct = 0
    model.eval()
    with torch.no_grad():
        for images, exp_results in valn_dataloader:
            images, exp_results = images.to(device), exp_results.to(device)

            log_pred_results = model(images)

            loss = criterion(log_pred_results, exp_results)
            total_valn_loss += loss.item()

            predicted_results = torch.exp(log_pred_results)
            top_probs, top_results = predicted_results.topk(1, dim=1)
            correct_results = top_results == exp_results.view(*top_results.shape)
            total_correct += correct_results.sum().item()
   
    mean_trng_loss = total_trng_loss / len(trng_dataloader.dataset)
    mean_valn_loss = total_valn_loss / len(valn_dataloader.dataset)
    mean_correct = total_correct / len(valn_dataloader.dataset)

    trng_losses.append(mean_trng_loss)
    valn_losses.append(mean_valn_loss)

    duration = time.time() - start_time

    print("Epoch: {}/{}.. ".format(epoch + 1, max_epochs),
          "Training Loss: {:.5f}.. ".format(mean_trng_loss),
          "Validation Loss: {:.5f}.. ".format(mean_valn_loss),
          "Accuracy: {:.3f}.. ".format(mean_correct),
          "Duration: {:.1f}s".format(duration))

In [ ]:
plt.plot(trng_losses, label='Training loss')
plt.plot(valn_losses, label='Validation loss')
plt.legend(frameon=False);

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
total_correct = 0
model.to(device)
model.eval()
for images, exp_results in tstg_dataloader:
    images, exp_results = images.to(device), exp_results.to(device)

    with torch.no_grad():
        log_pred_results = model.forward(images)
    
    pred_results = torch.exp(log_pred_results)

    top_probs, top_results = pred_results.topk(1, dim=1)
    correct_results = top_results == exp_results.view(*top_results.shape)
    total_correct += correct_results.sum().item()
    
mean_correct = total_correct / len(tstg_dataloader.dataset)

print("Test accuracy: {:.3f}".format(mean_correct))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
checkpoint_file_path = 'flwr_class_model.pth'

In [ ]:
checkpoint = {'feature_model': 'densenet121',
              'hidden_layer_size': hidden_layer_size,
              'model_state': model.state_dict(),
              'idx_to_cat': idx_to_cat}
torch.save(checkpoint, checkpoint_file_path)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
checkpoint = torch.load(checkpoint_file_path, map_location=CPU_DEVICE)
model, optimizer, criterion = create_model_densenet121(hidden_layer_size=checkpoint['hidden_layer_size'])
model.load_state_dict(checkpoint['model_state'])
idx_to_cat = checkpoint['idx_to_cat']

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    shortest_side_size = 256
    crop_size = 224

    scale_factor = min(image.size) / shortest_side_size
    new_size = tuple([int(x / scale_factor) for x in image.size])
    image = image.resize(new_size)
    crop_width = (image.width - crop_size) // 2
    crop_height = (image.height - crop_size) // 2
    image = image.crop((crop_width, 
                        crop_height, 
                        image.width - crop_width, 
                        image.height - crop_height))

    np_image = np.array(image)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image / 255 - mean) / std
    np_image = np.transpose(np_image, (2, 0, 1))

    np_image = np_image.astype('float32')

    return np_image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
image_path = "flowers/test/5/image_05159.jpg"

with Image.open(image_path) as im:
    imshow(torch.from_numpy(process_image(im)))
    imshow(tstg_transforms(im))

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
# Modified function call to accept an image, rather than a path, to simplify other code.
def predict(image, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model.eval()
    model.to(device)
    with torch.no_grad():
        pred_results = torch.exp(model(image.unsqueeze(dim=0).to(device)))

    top_probs, top_results = pred_results.topk(topk, dim=1)

    top_probs = top_probs.squeeze_().tolist()
    top_cats = [idx_to_cat[x.item()] for x in top_results.squeeze_()]

    return top_probs, top_cats

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
def restore_image(image):   
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
        
    return image

In [ ]:
def show_image_prediction(image, top_probs, top_cats, actual_name=None):
    top_cat_names = [cat_to_name[i] for i in top_cats]

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(restore_image(image))
    ax1.axis('off')
    if actual_name != None:
        ax1.set_title(actual_name)
    ax2.barh(np.arange(5), top_probs)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(len(top_cat_names)))
    ax2.set_yticklabels(top_cat_names)
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [ ]:
#image_path = "flowers/valid/96/image_07670.jpg" # camellia
#image_path = "flowers/valid/42/image_05712.jpg" # daffodil
#image_path = "flowers/valid/16/image_06671.jpg" # globe-flow
image_path = "flowers/valid/8/image_03313.jpg" # bird of paradise

with Image.open(image_path) as im:
    image = tstg_transforms(im)

top_probs, top_cats = predict(image, model)
show_image_prediction(image, top_probs, top_cats)

In [ ]:
images, exp_results = iter(tstg_dataloader).next()
actual_name = cat_to_name[idx_to_cat[exp_results[0].item()]]
top_probs, top_cats = predict(images[0], model)
show_image_prediction(images[0], top_probs, top_cats, actual_name)

<font color='red'>**Reminder for Workspace users:** If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again. 
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.</font>

In [ ]:
# TODO remove .pth files or move it to a temporary `~/opt` directory in this Workspace